<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/gee_deforestation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
The goal of this notebook is to implement a python version of the JavaScript code [here](https://code.earthengine.google.com/73dcaa2cae2e4aad2da593ec6ab1af85), making use of python utility code available via the Earthshot Labs Forest repo. 

## Caveat about GEE utility code
This notebook uses code copied from [Github](https://github.com/earthshot-labs/forest/blob/main/earth-engine/landtrendr/preprocess.py) which should be assembled into a package (e.g. earthshot) and installed via pip instead. For now, we'll just copy the code over, though.

In [2]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lAZqr9_5i4qseLvUU9XkehjlICFugHAY4ulzF6tw4Bw&tc=9kGX4UaVyxfme4kFsLHmrut6IGCQpP7ng3f6xTjzEqA&cc=k1yS4rNe1iXI7rfVBCoIpIFokf5HhHBnP0U7ke6XyqM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qjFBRUF1eFoGIBYwyIAcH9UJbPITaQbJASHuEJnho5X_ekmgjwzFHE

Successfully saved authorization token.


In [4]:
## Code from preprocess.py

from typing import List

INDEX_DICT = {
    "NDVI": {"dist_dir": -1, "bands": ["B4", "B3"]}
    # "NBR": {"dist_dir": -1, "bands": ["B4", "B7"]},
}


def _extract_and_append_date(image: ee.Image, input_list: ee.List) -> ee.List:
    """Given an ee.Image and an ee.List, append the image's date to the list."""
    date = image.date()
    return ee.List(input_list).add(ee.Date(date))


def _create_yearly_list(collection: ee.ImageCollection) -> ee.List:
    """Given an ee.ImageCollection, return an ee.List containing all the (unique)
    years present in the collection."""
    dates = collection.iterate(_extract_and_append_date, ee.List([]))

    years = ee.List(dates).map(lambda date: ee.Date(date).get("year")).distinct().sort()

    return years


def _extract_medoid_image(
    year: int,
    collection: ee.ImageCollection,
    start_day: str = "06-20",
    end_day: str = "09-10",
) -> ee.Image:
    """Given a year of interest, an ee.ImageCollection, and boundaries of start
    and end days, return an ee.Image which is the medoid image for the year in
    question. Distance calculated using the standard Euclidean norm across all
    6 TM-equivalent bands. Question: is it possible to do better, for the purposes
    of a deforestation analysis, by applying a higher weight to NIR and SWIR bands?
    Parameters
    ----------
    year: int
      The year for which the medoid should be calculated.
    collection: ee.ImageCollection
      The collection whose medoid should be calculated.
    start_day: str
      The first day, inclusive, to consider in calculating the medoid. Formatted
      as 'mm-dd'.
    end_day: str
      The last day, inclusive, to consider in calculating the medoid. Formatted as
      'mm-dd'.
    Returns
    -------
    ee.Image
      An ee.Image containing the pixel-wise medoid for the year in question.
    """
    start_m, start_d = start_day.split("-")
    end_m, end_d = end_day.split("-")

    start_date = ee.Date.fromYMD(ee.Number(year), int(start_m), int(start_d))
    end_date = ee.Date.fromYMD(ee.Number(year), int(end_m), int(end_d)).advance(
        1, "day"
    )
    filtered_collection = collection.filterDate(start_date, end_date)
    empty_col = ee.ImageCollection([ee.Image([0,0,0,0,0,0]).mask(ee.Image(0))]);
    non_empty = filtered_collection.toList(1).length().gt(0)
    final_collection = ee.ImageCollection(ee.Algorithms.If(non_empty, filtered_collection, empty_col))
    median = final_collection.median()

    def _euclidean_distance(image: ee.Image) -> ee.Image:
        distance = ee.Image(image).subtract(median).pow(ee.Image.constant(2))
        return distance.reduce("sum").addBands(image)

    distance_from_median = final_collection.map(_euclidean_distance)
    return (
        ee.ImageCollection(distance_from_median)
        .reduce(ee.Reducer.min(7))
        .set("system:time_start", ee.Date.fromYMD(year, 8, 1).millis())
    )


def _generate_medoid_collection(
    collection: ee.ImageCollection, start_day: str, end_day: str
) -> ee.ImageCollection:
    """Given an ee.ImageCollection and bounds on the start and end days, compute
    an ee.ImageCollection which contains a medoid image for each year present in
    the input collection.
    Parameters
    ----------
    collection: ee.ImageCollection
      The collection whose medoid should be calculated.
    start_day: str
      The first day, inclusive, to consider in calculating the medoid. Formatted
      as 'mm-dd'.
    end_day: str
      The last day, inclusive, to consider in calculating the medoid. Formatted as
      'mm-dd'.
    Returns
    -------
    ee.ImageCollection
      A collection of yearly medoid images for the provided collection.
    """

    years = _create_yearly_list(collection)

    def _extract_medoid(year):
        return _extract_medoid_image(year, collection, start_day, end_day)

    images = years.map(_extract_medoid)
    return ee.ImageCollection.fromImages(images)


def _mask_landsat_sr(image: ee.Image) -> ee.Image:
    """Apply a mask to a Landsat image to filter out water, cloud, snow, and cloud
    shadow pixels."""
    qa_band = image.select("pixel_qa")
    # Bits 2, 3, 4, and 5 of pixel_qa band of Landsat are water, cloud shadow, snow,
    # and cloud, respectively. Define bitmasks for these entries below:
    water_bit_msk = 1 << 2
    cloud_shadow_bit_msk = 1 << 3
    snow_bit_msk = 1 << 4
    cloud_bit_msk = 1 << 5

    qa_mask = (
        qa_band.bitwiseAnd(water_bit_msk)
        .eq(0)
        .And(qa_band.bitwiseAnd(cloud_shadow_bit_msk).eq(0))
        .And(qa_band.bitwiseAnd(snow_bit_msk).eq(0))
        .And(qa_band.bitwiseAnd(cloud_bit_msk).eq(0))
    )

    return image.updateMask(qa_mask)


def _prepare_images(
    image: ee.Image, input_bands: List[str], output_bands: List[str]
) -> ee.Image:
    """Takes an ee.Image object and a list of input and output bands; this function
    resamples the image using bilinear resampling, applies a QA mask, and returns
    an ee.Image with the output bands selected and with the system:time_start
    field set from input image.
    Intended to be used via partial execution for TM and OLI collections.
    Parameters
    ----------
    image: ee.Image
      The image to prepare.
    input_bands: List[str]
      The bands to select from the image.
    output_bands: List[str]
      The labels for the selected bands in the output image.
    Returns
    -------
    ee.Image
      The input image, resampled, with QA mask applied, and with bands renamed.
    """

    resampled_image = image.resample("bilinear").set(
        "system:time_start", image.get("system:time_start")
    )

    return _mask_landsat_sr(resampled_image).select(input_bands, output_bands)


def _build_TM_collection(
    sensor: str,
    aoi: ee.Geometry,
    start_year: int = 1985,
    start_day: int = "06-20",
    end_year: int = 2020,
    end_day: int = "09-10",
) -> ee.ImageCollection:
    """Given the sensor, area of interest, and date boundaries, return a collection
    of TM (or ETM+) Landsat images. Intended for use on Landsat 5 and Landsat 7
    images.
    Parameters
    ----------
    sensor: str
      The sensor to use -- expects one of 'LT05' or 'LE07'.
    aoi: ee.Geometry
      The area of interest for the collection. This will be used in a filterBounds
      call on the collection to reduce size.
    start_year: int
      The first year (inclusive) to get data.
    start_day: str
      The first day (inclusive) to get data. Formatted as 'mm-dd'.
    end_year: int
      The last year (inclusive) to get data.
    end_day: str
      The last day (inclusive) to get data. Formatted as 'mm-dd'.
    Returns
    -------
    ee.ImageCollection
      The filtered collection.
    """

    def _prepare_TM(image: ee.Image) -> ee.Image:
        return _prepare_images(
            image=image,
            input_bands=["B1", "B2", "B3", "B4", "B5", "B7"],
            output_bands=["B1", "B2", "B3", "B4", "B5", "B7"],
        )

    collection = (
        ee.ImageCollection("LANDSAT/" + sensor + "/C01/T1_SR")
        .filterBounds(aoi)
        .filterDate(
            ee.Date(str(start_year) + "-" + start_day),
            ee.Date(str(end_year) + "-" + end_day).advance(1, "day"),
        )
    )
    return collection.map(_prepare_TM)


def _build_OLI_collection(
    sensor: str,
    aoi: ee.Geometry,
    start_year: int = 1985,
    start_day: int = "06-20",
    end_year: int = 2020,
    end_day: int = "09-10",
) -> ee.ImageCollection:
    """Given the sensor, area of interest, and date boundaries, return a collection
    of TM-equivalent Landsat images from an OLI Landsat collection. Intended for
    use on Landsat 8 images.
    This function applies a linear transformation to harmonize Landsat 7 (ETM) and
    Landsat 8 (OLI) images. Transformation developed by Roy et al. and implemented
    by Justin Braaten at:
    https://github.com/eMapR/LT-GEE/blob/master/scripts/python/lt_gee_bap_test.py#L60
    Parameters
    ----------
    sensor: str
      The sensor to use -- expects 'LC08'.
    aoi: ee.Geometry
      The area of interest for the collection. This will be used in a filterBounds
      call on the collection to reduce size.
    start_year: int
      The first year (inclusive) to get data.
    start_day: str
      The first day (inclusive) to get data. Formatted as 'mm-dd'.
    end_year: int
      The last year (inclusive) to get data.
    end_day: str
      The last day (inclusive) to get data. Formatted as 'mm-dd'.
    Returns
    -------
    ee.ImageCollection
      The filtered collection with OLI bands scaled and renamed to TM-equivalents.
    """

    def _harmonization_Roy(image: ee.Image) -> ee.Image:
        """Taken verbatim from Justin Braaten's implementation at:
        https://github.com/eMapR/LT-GEE/blob/b0e92a0c198bdd1a794e1e9b8f4db8fc7fa06054/scripts/python/lt_gee_bap_test.py#L60"""
        slopes = ee.Image.constant([0.9785, 0.9542, 0.9825, 1.0073, 1.0171, 0.9949])
        intercepts = ee.Image.constant(
            [-0.0095, -0.0016, -0.0022, -0.0021, -0.0030, 0.0029]
        )

        return (
            image.subtract(intercepts.multiply(10000))
            .divide(slopes)
            .toShort()
            .set("system:time_start", image.get("system:time_start"))
        )

    def _prepare_OLI(image: ee.Image) -> ee.Image:
        return _prepare_images(
            image=image,
            input_bands=["B2", "B3", "B4", "B5", "B6", "B7"],
            output_bands=["B1", "B2", "B3", "B4", "B5", "B7"],
        )

    collection = (
        ee.ImageCollection("LANDSAT/" + sensor + "/C01/T1_SR")
        .filterBounds(aoi)
        .filterDate(
            ee.Date(str(start_year) + "-" + start_day),
            ee.Date(str(end_year) + "-" + end_day).advance(1, "day"),
        )
    )
    return collection.map(_prepare_OLI).map(_harmonization_Roy)


def _build_combined_Landsat(
    aoi: ee.Geometry,
    start_year: int = 1985,
    start_day: int = "06-20",
    end_year: int = 2020,
    end_day: int = "09-10",
) -> ee.ImageCollection:
    """
    Given an area of interest and date bounds, return a collection containing
    Landsat 5, 7, and 8 images. A linear rescaling is applied to Landsat 8 bands,
    which are from the Operational Land Imager (OLI) instrument, to convert them
    to Thematic Mapper-equivalent values.
    Parameters
    ----------
    aoi: ee.Geometry
      The area of interest for the collection.
    start_year: int
      The first year (inclusive) to get data.
    start_day: str
      The first day (inclusive) to get data. Formatted as 'mm-dd'.
    end_year: int
      The last year (inclusive) to get data.
    end_day: str
      The last day (inclusive) to get data. Formatted as 'mm-dd'.
    Returns
    -------
    ee.ImageCollection
      The TM-equivalent bands from Landsat 5, 7, and 8 for the time period and
      region of interest.
    """
    landsat5 = _build_TM_collection(
        sensor="LT05",
        aoi=aoi,
        start_year=start_year,
        start_day=start_day,
        end_year=end_year,
        end_day=end_day,
    )

    landsat7 = _build_TM_collection(
        sensor="LE07",
        aoi=aoi,
        start_year=start_year,
        start_day=start_day,
        end_year=end_year,
        end_day=end_day,
    )

    landsat8 = _build_OLI_collection(
        sensor="LC08",
        aoi=aoi,
        start_year=start_year,
        start_day=start_day,
        end_year=end_year,
        end_day=end_day,
    )

    return ee.ImageCollection(landsat5.merge(landsat7).merge(landsat8))


def build_SR_collection(
    aoi: ee.Geometry, start_year: int, start_day: str, end_year: int, end_day: str
) -> ee.ImageCollection:
    """
    Given an area of interest and date bounds, return a collection containing a
    yearly medoid for each image in the date range using Landsat 5, 7, and 8
    images.
    Python attempt at implementing:
    https://emapr.github.io/LT-GEE/api.html#buildsrcollection
    Parameters
    ----------
    aoi: ee.Geometry
      The area of interest for the collection.
    start_year: int
      The first year (inclusive) to get data.
    start_day: str
      The first day (inclusive) to get data. Formatted as 'mm-dd'.
    end_year: int
      The last year (inclusive) to get data.
    end_day: str
      The last day (inclusive) to get data. Formatted as 'mm-dd'.
    Returns
    -------
    ee.ImageCollection
      The collection of yearly medoid images.
    """

    combined_landsat = _build_combined_Landsat(
        aoi, start_year, start_day, end_year, end_day
    )
    return _generate_medoid_collection(combined_landsat, start_day, end_day).select(
        [1, 2, 3, 4, 5, 6], ["B1", "B2", "B3", "B4", "B5", "B7"]
    )


def build_LT_collection(
    collection: ee.ImageCollection, index: str, ftv_list: List[str]
) -> ee.ImageCollection:
    """
    Given a surface reflectance collection produced by build_SR_collection,
    the spectral index of interest, and a list of bands to include, produce
    a collection for Landtrendr.
    The first band will be the index of interest, scaled so that an increase
    in the band value indicates vegetation loss.
    Python attempt at implementing:
    https://emapr.github.io/LT-GEE/api.html#buildltcollection
    Parameters
    ----------
    collection: ee.ImageCollection
        The collection to prepare for Landtrendr.
    index: str
        The spectral index to use. Supported values are 'NDVI' and 'NBR'.
    ftv_list: List[str]
        Additional bands to include in the collection.
    Returns
    -------
    ee.ImageCollection
      The collection of yearly medoid images.
    """
    try:
        index_info = INDEX_DICT[index]
    except KeyError:
        if index in ["NDSI", "NDMI", "TCB", "TCG", "TCW", "TCA", "NBR2"]:
            # If users provide a valid spectral index, they should get a more helpful
            # exception explaining the source of the issue than just a random KeyError
            raise NotImplementedError(
                f"The index '{index}' is not currently supported. Supported indices are: "
                + ", ".join(list(INDEX_DICT.keys()))
            )
        else:
            raise RuntimeError(
                f"The value '{index}' was not recognized as a standard spectral index."
            )

    dist_dir = index_info["dist_dir"]
    index_bands = index_info["bands"]
    ftv_bands = ["ftv_" + band.lower() for band in ftv_list]

    # This function should support a spectral index as a FTV, but a clever way to
    # do this without mixing client- and server-side functions currently eludes me,
    # so I am going to skip it for now.
    def _format_images(image):
        idx = (
            image.normalizedDifference(index_bands)
            .rename(index)
            .multiply(dist_dir * 1000)
        )
        ftv_values = image.select(ftv_list, ftv_bands)
        return idx.addBands(ftv_values).set(
            "system:time_start", image.get("system:time_start")
        )

    return collection.map(_format_images)

In [5]:
## Code from postprocess.py

INDEX_DICT = {
    "NDVI": {"dist_dir": -1, "bands": ["B4", "B3"]}
    # "NBR": {"dist_dir": -1, "bands": ["B4", "B7"]},
}


def get_segment_data(lt: ee.Image, index: str, right: bool = False) -> ee.Image:
    """
    Given a Landtrendr output, the index of interest, and a boolean flag
    denoting whether or not to reorient inverted index values, return an ee.Image
    object containing information on the segments during which Landtrendr
    detected losses in forest cover.
    Parameters
    ----------
    lt: ee.Image
        The Landtrendr output as an ee.Image object.
    index: str
        The spectral index for which Landtrendr was run.
    right: boolean
        Whether or not to correct the orientation of the index if it has been
        inverted in the process of preparing the Landtrendr collection (see:
        https://emapr.github.io/LT-GEE/api.html#buildltcollection)
    Returns
    -------
    ee.Image
        An image with information on the loss segments as array-valued pixels.
    Python attempt at implementing:
    https://emapr.github.io/LT-GEE/api.html#getsegmentdata"""
    lt_output = lt.select("LandTrendr")
    vertex_mask = lt_output.arraySlice(0, 3, 4)
    vertices = lt_output.arrayMask(vertex_mask)

    try:
        index_info = INDEX_DICT[index]
    except KeyError:
        if index in ["NDSI", "NDMI", "TCB", "TCG", "TCW", "TCA", "NBR"]:
            # If users provide a valid spectral index, they should get a more helpful
            # exception explaining the source of the issue than just a random KeyError
            raise NotImplementedError(
                f"The index '{index}' is not currently supported. Supported indices are: "
                + ", ".join(list(INDEX_DICT.keys()))
            )
        else:
            raise RuntimeError(
                f"The value '{index}' was not recognized as a standard spectral index."
            )

    if right:
        dist_dir = index_info["dist_dir"]
    else:
        dist_dir = 1

    left = vertices.arraySlice(1, 0, -1)
    right = vertices.arraySlice(1, 1, None)
    start_year = left.arraySlice(0, 0, 1)
    start_val = left.arraySlice(0, 2, 3).multiply(dist_dir)
    end_year = right.arraySlice(0, 0, 1)
    end_val = right.arraySlice(0, 2, 3).multiply(dist_dir)

    dur = end_year.subtract(start_year)
    mag = end_val.subtract(start_val).multiply(dist_dir)
    rate = mag.divide(dur)
    dsnr = mag.divide(lt.select("rmse"))

    seg_info = (
        ee.Image.cat(
            [start_year.add(1), end_year, start_val, end_val, mag, dur, rate, dsnr]
        )
        .toArray(0)
        .updateMask(vertex_mask.mask())
    )

    # FIXME: add filter on magnitude here to restrict to only loss segments

    return seg_info


def extract_deforestation_events(
    LT_segments: ee.Image, start_year: int, end_year: int, dsnr_threshold: float
) -> ee.Image:
    """
    Given an image with information about loss segments, assumed to be the
    output of get_segment_data, bounds on start and end year, and a threshold
    on the disturbance signal-to-noise ratio (DSNR), filter to only events
    within the years of interest which pass the DSNR threshold.
    Note: the decision to extract the most recent event, rather than the
    selecting the largest or using a different selection mechanism, is made
    because this was written with the goal of identifying deforestation events
    followed by a recovery period.
    Parameters
    ----------
    LT_segments: ee.Image
        Landtrendr segment data, assumed to be the output of get_segment_data
    start_year: int
        The first year to consider for deforestation events.
    end_year: int
        The last year to consider for deforestation events.
    dsnr_threshold: float
        The threshold on the disturbance signal-to-noise ratio (DSNR) for the
        deforestation segments.
    Returns
    -------
    ee.Image
        The input image filtered to the most recent event in each pixel which
        passes the given threshold.
    """
    start_years = LT_segments.arraySlice(0, 0, 1)
    end_years = LT_segments.arraySlice(0, 1, 2)
    dsnr = LT_segments.arraySlice(0, 7, 8)
    mask = (
        start_years.gte(ee.Image(start_year))
        .And(end_years.lte(ee.Image(end_year)))
        .And(dsnr.gte(ee.Image(dsnr_threshold)))
    )

    masked_segments = LT_segments.arrayMask(mask)

    # Extract the most recent segments for each pixel
    # factor of -1 to flip delta, since arraySort is ascending
    sort_by = masked_segments.arraySlice(0, 0, 1).toArray(0).multiply(-1)
    segments_sorted = masked_segments.arraySort(sort_by)
    return segments_sorted.arraySlice(1, 0, 1)


def extract_deforested_regions(
    LT_result: ee.Image,
    index: str,
    start_year: int,
    end_year: int,
    dsnr_threshold: float,
):
    """
    Given the output of Landtrendr as an image, the index of interest, bounds on
    the beginning and end of the period to identify events, and a threshold on
    the DSNR, return a flattened image containing information about the most
    recent deforestation event in each pixel which passes the specified DSNR
    threshold.
    Note: the decision to extract the most recent event, rather than the
    selecting the largest or using a different selection mechanism, is made
    because this was written with the goal of identifying deforestation events
    followed by a recovery period.
    Parameters
    ----------
    LT_segments: ee.Image
        Landtrendr segment data, assumed to be the output of get_segment_data
    index: str
        The spectral index for which Landtrendr was run.
    start_year: int
        The first year to consider for deforestation events.
    end_year: int
        The last year to consider for deforestation events.
    dsnr_threshold: float
        The threshold on the disturbance signal-to-noise ratio (DSNR) for the
        deforestation segments.
    """
    LT_segments = get_segment_data(LT_result, index, True)
    events = extract_deforestation_events(
        LT_segments, start_year, end_year, dsnr_threshold
    )

    flattened_image = ee.Image.cat(
        events.arraySlice(0, 0, 1).arrayProject([1]).arrayFlatten([["yod"]]),
        events.arraySlice(0, 1, 2).arrayProject([1]).arrayFlatten([["endYr"]]),
        events.arraySlice(0, 2, 3).arrayProject([1]).arrayFlatten([["startVal"]]),
        events.arraySlice(0, 3, 4).arrayProject([1]).arrayFlatten([["endVal"]]),
        events.arraySlice(0, 4, 5).arrayProject([1]).arrayFlatten([["mag"]]),
        events.arraySlice(0, 5, 6).arrayProject([1]).arrayFlatten([["dur"]]),
        events.arraySlice(0, 6, 7).arrayProject([1]).arrayFlatten([["rate"]]),
        events.arraySlice(0, 7, 8).arrayProject([1]).arrayFlatten([["dsnr"]]),
    )

    return flattened_image

In [55]:
## Input geometry of interest here as an ee.Geometry object -- this ee.Geometry 
## object is expected by the functions below.
#UL -41.87, -17.89, -39.87, -19.89
#UR -39.97, -17.89, -37.97, -19.89
#LL -41.87, -19.79, -39.87, -21.79
#LR NA NA NA NA


geom = ee.Geometry.Rectangle([[-41.87, -19.79], [-39.87, -21.79]])

In [56]:
start_year = 1985
end_year = 2020
start_day = '06-20'
end_day = '09-10'
aoi = geom
dsnr_threshold = 2.0
mask_these = ['cloud', 'shadow', 'snow', 'water']
index = 'NDVI'
ftv_list = [
    # 'NBR', # normalized burn ratio 
    'B4', # LS band 4, red
    'B5', # LS band 5, near-infrared
]
landtrendr_run_params = {
    "maxSegments": 6,
    "spikeThreshold": 0.9,
    "vertexCountOvershoot": 3,
    "preventOneYearRecovery": True,
    "recoveryThreshold": 0.25,
    "pvalThreshold": 0.05,
    "bestModelProportion": 0.75,
    "minObservationsNeeded": 6
}

In [57]:
landsat_surface_reflectance = build_SR_collection(
    aoi=aoi,
    start_year=start_year,
    start_day=start_day,
    end_year=end_year,
    end_day=end_day
)

landsat_landtrendr_collection = build_LT_collection(
    collection=landsat_surface_reflectance, index=index, ftv_list=ftv_list
)

landtrendr_run_params["timeSeries"] = landsat_landtrendr_collection

In [58]:
LT_result = ee.Algorithms.TemporalSegmentation.LandTrendr(**landtrendr_run_params)

In [59]:
deforested_regions = extract_deforested_regions(
    LT_result=LT_result, 
    index=index, 
    start_year=start_year, 
    end_year=end_year, 
    dsnr_threshold=dsnr_threshold
)

In [11]:
# Finally, export the deforested_regions image (ee.Image object) to a location of your choosing.

In [60]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': deforested_regions,
    'description': 'landtrendr_Espirito_Santos_LL',
    'folder':'Example_folder',
    'scale': 30,
    'region': geom.getInfo()['coordinates']
})
task.start()

In [29]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: 7DFHU737OFMX5F3NQF6WMRLP).


KeyboardInterrupt: ignored

In [14]:
landsat_surface_reflectance = build_SR_collection(
    aoi=aoi,
    start_year=start_year,
    start_day=start_day,
    end_year=end_year,
    end_day=end_day
)

In [15]:
# Export the image, specifying scale and region.
# output_image = ee.merge(landsat_surface_reflectance)
# ee.batch.Export.ImageCollection
# ee.batch.Download
task = ee.batch.Download.ImageCollection.toDrive(**{
    'image': landsat_surface_reflectance,
    'description': 'Surface_Reflectance',
    'folder':'Example_folder',
    'scale': 1000,
    'region': geom.getInfo()['coordinates']
})
task.start()

AttributeError: ignored

In [ ]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
pip install gee2drive

In [ ]:
import gee2drive as g2d

In [ ]:
g2d.